# 41934 Advanced BIM - A3
#### **Authors**:
Rihards Zamoidiks, s186415  
Eliza Anna Grinberga, s223262


Welcome to this commentary notebook!  
Here, we provide a more detailed narrative to accompany the script developed for our tool in A3, enhancing your learning experience as you explore the tool's functionality and implementation.  

**NOTE:** This script has minor, non-operational changes compared to the original Python script, to ensure compatibility with Jupyter Notebooks.

As before, we start by importing all the necessary Python libraries essential for the operation of this tool:

In [9]:
from pathlib import Path
import ifcopenshell
import ifcopenshell.util.element
import ifcopenshell.api
import sys

We proceed by establishing functions to manage some of the more repetitive tasks required by the tool.

The very first one, named `loadFile`, an exact copy of the one employed in A2, is responsible for loading an IFC file. When you run this function, it will ask you to input the name of the IFC file you want to load. If you press Enter without typing anything, it will default to a file named 'LLYN - STRU.ifc', which has been commonly used during this course.

**Note:** To ensure successful loading, place the IFC file in the designated `model` folder. 

The function checks if the specified file exists. If it does, it loads the file and returns two pieces of information: the loaded model and its name. If the file is not found, it gives an error message.

Here's how the function looks in Python:

In [10]:
def loadFile():
    # Specify the name of the IFC model
    modelname = input("Enter the IFC model name (or press Enter for default 'LLYN - STRU'): ")
    
    # Set a default value if the user didn't provide input
    if not modelname:
        modelname = 'LLYN - STRU'
    
    try:
        dir_path = Path.cwd()
        model_url = Path.joinpath(dir_path, 'model', modelname).with_suffix('.ifc')
        model = ifcopenshell.open(model_url)
    except OSError:
        try:
            # If file not found, bpy import will produce an error, if script is run outside of Blender
            import bpy
            model_url = Path.joinpath(Path(bpy.context.space_data.text.filepath).parent, 'model', modelname).with_suffix('.ifc')
            model = ifcopenshell.open(model_url)
        except OSError:
            print(f'ERROR: please check your model folder : {model_url} does not exist')
    
    # Load the IFC model
    model = ifcopenshell.open(model_url)
    
    return model, modelname

This function, `createMaterialProp`, initializes a property list and prompts the user to input two values, specifically for material type and strength class. It ensures the user provides both inputs before returning the list of properties.

In [11]:
def createMaterialProp():
    # Initialize a property list
    prop = []

    while len(prop) < 2:
        if len(prop) == 0:
            user_input = input("\nEnter Material: ")
        else:
            user_input = input("\nEnter Strength class: ")

        prop.append(user_input)
    
    return prop

The `assignProperty` function ensures the assignment of a newly created PropertySet to specific types of structural elements within an IFC model. The function begins by calling `createMaterialProp()` to gather user input as covered above. Subsequently, it retrieves the IFC elements of the specified type, and for each element, it adds the new PropertySet and populates it with the provided material properties. Finally, it notifies the user that the material properties have been successfully added to the specified elements.

In [12]:
def assignProperty(ifc_file, element, propName):
    
    propValue = createMaterialProp()
    
    # Retrieve IFC elements of the specified type
    str_elements = model.by_type(str(element))
    
    # Check if the IFC model contains specified structural elements
    if len(str_elements) != 0:
       # Iterate through elements of the specified type
        for str_element in str_elements:
            pset = ifcopenshell.api.run("pset.add_pset", ifc_file, product=str_element, name=propName)
            ifcopenshell.api.run("pset.edit_pset", ifc_file, pset=pset, properties={"Material": propValue[0], "Strength class": propValue[1]})
    
    print('Material property -', propValue[0], ":", propValue[1], "has been added to all ", element, "elements.")

The `saveFile` function facilitates the process of saving modifications made to the IFC model. It takes the modified model and the original model name as parameters, constructs a new file path for the modified model, saves the file to the specified path and then notifies the user that changes have been saved.

In [13]:
def saveFile(model, modelname):
    path = './output/' + "modified_" + modelname + '.ifc'
    model.write(path)
    print("\nSaving changes to ", path, "and exiting the menu.")

With the essential functions now in place, we turn to the main portion of the script. This primarily involves a single while loop, complemented by the initial file reading process.

This following part of the script establishes a user interface through a while loop, providing a menu for selecting different types of structural elements within an IFC file. The user can choose between IfcColumn, IfcWall, IfcBeam, IfcSlab and the option to exit the script. Upon selecting a type, the `assignProperty` function is called to manage the assignment of material properties to the chosen elements. The loop then prompts the user for repetition or exit, ensuring a user-friendly and interactive experience. Additionally, it saves the modified IFC file when the user chooses to exit the script.

In [14]:
# Load the IFC model
model, modelname = loadFile()  

# Initialize operation flag
flag = True

while flag:
    # Display the menu
    print("\nSelect an option:")
    print("1. IfcColumn")
    print("2. IfcWall")
    print("3. IfcBeam")
    print("4. IfcSlab")
    print("5. Exit")

    # Get user input
    choice = input("\nEnter your choice: ")

    if choice == "1":
        
        element = "IfcColumn"
        print("You selected ", element)
        assignProperty(model, element, "Pset_MaterialStructural")
            
    elif choice == "2":
        
        element = "IfcWall"
        print("You selected ", element)
        assignProperty(model, element, "Pset_MaterialStructural")
        
    elif choice == "3":
        
        element = "IfcBeam"
        print("You selected ", element)
        assignProperty(model, element, "Pset_MaterialStructural")
        
    elif choice == "4":
        
        element = "IfcSlab"
        print("You selected ", element)
        assignProperty(model, element, "Pset_MaterialStructural")
        
    elif choice == "5":
         saveFile(model, modelname)
         break
    else:
        print("\nInvalid choice. Please select a valid option.")
 
    while flag:
        repeat = input("\nDo you want to repeat the script? (yes/no): ")
        if repeat.lower() == "yes":
            break  # Break out of the inner loop to repeat the entire script
        elif repeat.lower() == "no":
            saveFile(model, modelname)
            flag = False
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")

Enter the IFC model name (or press Enter for default 'LLYN - STRU'):  LLYN - STRU



Select an option:
1. IfcColumn
2. IfcWall
3. IfcBeam
4. IfcSlab
5. Exit



Enter your choice:  1


You selected  IfcColumn



Enter Material:  Concrete

Enter Strength class:  C40/45


Material property - Concrete : C40/45 has been added to all  IfcColumn elements.



Do you want to repeat the script? (yes/no):  no



Saving changes to  ./output/modified_LLYN - STRU.ifc and exiting the menu.
